### 11/3, random-seedのみを変えた10例について検討 + plotlyチュートリアル

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
log_df = pd.read_csv("./experiment/experiment_log.csv")
log_df = log_df.loc[log_df["run_id"].map(lambda x: "cat4_batch4_acc8_seed" in x), :]

In [4]:
log_df["model"] = log_df["run_id"].map(lambda x: x.split("_")[0])

In [5]:
log_df.loc[:, ["Single_Public_LB", "all_valid_metric", "mean_valid_metric", "run_id", "seed"]]

,Single_Public_LB,all_valid_metric,mean_valid_metric,run_id,seed
106,0.755,0.737342,0.737031,roberta-large_cat4_batch4_acc8_seed0,0.0
107,0.703,0.691152,0.691405,roberta-large_cat4_batch4_acc8_seed1,1.0
108,0.651,0.662116,0.661627,roberta-large_cat4_batch4_acc8_seed2,2.0
109,0.705,0.708520,0.709560,roberta-large_cat4_batch4_acc8_seed3,3.0
110,0.685,0.593002,0.534125,roberta-large_cat4_batch4_acc8_seed4,4.0
111,0.752,0.723005,0.722984,roberta-large_cat4_batch4_acc8_seed5,5.0
112,0.760,0.711921,0.714895,roberta-large_cat4_batch4_acc8_seed6,6.0
113,0.764,0.703518,0.699871,roberta-large_cat4_batch4_acc8_seed7,7.0
114,0.751,0.689320,0.682540,roberta-large_cat4_batch4_acc8_seed8,8.0
115,0.695,0.629091,0.568414,roberta-large_cat4_batch4_acc8_seed9,9.0


In [6]:
describe_df = np.round(log_df.groupby("model").agg(["mean", "std"])["all_valid_metric"].copy(), 3)

/tmp/ipykernel_1564820/417702080.py:1: FutureWarning: ['run_id', 'output_path', 'train_data', 'use_amp', 'model_name', 'model_custom_header', 'mixout', 'learning_rate', 'scheduler_name', 'save_checkpoint'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  describe_df = np.round(log_df.groupby("model").agg(["mean", "std"])["all_valid_metric"].copy(), 3)


In [37]:
describe_df.iloc[[0, 1, 3], :]

,mean,std
model,,
bert-base-whm,0.527,0.028
bert-large,0.539,0.036
xlm-roberta-large,0.662,0.034


In [8]:
describe_df.iloc[[0, 1, 3]:, :].index.tolist()

['bert-large', 'roberta-large', 'xlm-roberta-large']

### 主な気づき
* update_annotationsは打った時点で存在するannotationに作用する
    * subplot_titleに対するupdate_annotationは後ろからaddするannotationには聞いてない

In [39]:
# 基本イメージはfig, axと同じ。Figureを定義してAxesを後から足していく --
fig = make_subplots(
    rows=1, cols=2, 
    column_width=[0.5, 0.5], specs=[[{}, {"type": "domain"}]],
    subplot_titles=["乱数シードを変えたときのf1-scoreの箱ひげ図", "乱数シードを変えたときのモデルごとの平均・標準偏差"],
    )
fig.update_annotations(font=dict(size=20, color="gray"), y=1.04)

# add boxplots for each model -- 
colorlist = ["gray", "gray", "gray", "blue"]
boxplot_color_dict = {
    "gray": ("#a0a0a0", "#e0e0e0"),
    "blue": ("#7fbfff", "#e0efff"),
}
for model, color in zip(log_df["model"].unique(), colorlist):

    # nlp-waseda/roberta は一応隠そう --
    if model == "roberta-large": continue

    fig.add_trace(
        go.Box(
            x=log_df[log_df["model"] == model]["model"],
            y=log_df[log_df["model"] == model]["all_valid_metric"],
            width=0.59,
            boxmean=False,
            line=dict(color=boxplot_color_dict[color][0]),
            fillcolor=boxplot_color_dict[color][1],
        ),
        row=1, col=1
    )

fig.add_trace(
    go.Table(
        header=dict(
            values=["model"] + describe_df.iloc[[0, 1, 3], :].columns.tolist(),
            font=dict(size=18),
            height=42,
            align=["center", "center", "center"],
            fill_color="#add6ff",

        ),
        cells=dict(
            values=[describe_df.iloc[[0, 1, 3], :].index.tolist(), describe_df.iloc[[0, 1, 3], 0].values, describe_df.iloc[[0, 1, 3], 1].values],
            font=dict(size=16),
            height=32,
            align=["center", "center", "center"],
            fill_color=["#c1e0ff", "#dbedff", "#dbedff"],
        ),
        columnwidth=[1.4, 1, 1],
    ),
    row=1, col=2
)


# add explanation text --
fig.add_annotation(
    x=0.56, y=0.45, xref="paper",
    align="left", font=dict(size=16), showarrow=False, bgcolor="#f9ffff", xanchor="left",
    text=
        "【計算条件】<br>"\
        "・train_data: 5256<br>"
        "・stratified kfold: 5<br>"\
        "・train batch size: 4<br>"\
        "・gradient accumulation steps: 8              <br>"\
        "・learning rate: 1e-5, (with linear warmup)   <br>"\
        "・optimizer: AdamW<br>"\
        "・Out-of-Foldで計算したlabelを用いてf1-scoreを計算<br><br>"\
    )

fig.update_layout(
    # plot area --
    margin=dict(l=120, r=120, t=150, b=100), # facecolor部分の余白 --
    paper_bgcolor="#FFFFFF", # facecolor --
    width=1520, height=700, # graph area size --
    plot_bgcolor="#FFFFFF",

    # title and axis --
    title=dict(text="<b>乱数シードによるf1-scoreのバラつきの検討</b>", font=dict(size=28, color="gray"), xref="container", xanchor="center", x=0.5, y=0.94),
    xaxis=dict(title="model_name [str]", title_font=dict(size=26), tickfont=dict(size=18), color="gray", linecolor="gray", linewidth=2),
    yaxis=dict(title="f1-score [-]", title_font=dict(size=26), tickfont=dict(size=18), color="gray", linecolor="white", linewidth=4, gridcolor="lightgray", range=[0.44, 0.72]),

    showlegend=False,

    # hover --
    hovermode="closest",
    hoverlabel=dict(bgcolor="lightblue", font_size=14)
)
fig.show()